# Buurman 2010 Frequency Index Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jwellik/csav_seismology/blob/main/dev/buurman_2010_frequency_index.ipynb)

This notebook computes and visualizes Frequency Index (FI) values from seismic catalogs and waveforms, following the methodology of Buurman and West (2010).

**Two workflows available:**
1. **Compute FI from catalog + datasource**: Automatically download waveforms and calculate FI
2. **Load pre-computed data**: Use your own FI values, magnitudes, and waveforms

---

## 🚀 Google Colab Setup

**If running in Google Colab**, run the setup cell below to:
- Mount Google Drive (optional, for accessing files from Drive)
- Enable file uploads
- Set up the working directory

**If running locally**, skip the setup cell and proceed to installation.

In [ ]:
# Cell 0: Google Colab Setup (Skip if running locally)
# ============================================================================
# This cell handles Google Colab-specific setup
# ============================================================================

import os

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ Running locally (not Colab)")

if IN_COLAB:
    # Mount Google Drive (optional - uncomment if you want to access files from Drive)
    # from google.colab import drive
    # drive.mount('/content/drive')
    # print("✓ Google Drive mounted at /content/drive")
    
    # Create a directory for uploaded files
    os.makedirs('/content/data', exist_ok=True)
    print("✓ Created /content/data directory for uploaded files")
    
    # Install ipywidgets for file upload (if not already installed)
    # !pip install -q ipywidgets
    
    print("\n📁 To upload files:")
    print("  1. Use the file upload widget in the next cell, OR")
    print("  2. Mount Google Drive and place files in Drive, OR")
    print("  3. Use wget/curl to download files from URLs")
else:
    print("\n📁 Using local file system")
    print("  Set file paths in Cell 3 (Configuration)")

# Set working directory for Colab
if IN_COLAB:
    WORK_DIR = '/content/data'
else:
    WORK_DIR = os.getcwd()

print(f"\n✓ Working directory: {WORK_DIR}")

In [ ]:
# Cell 0b: File Upload Widget (Google Colab only)
# ============================================================================
# Use this cell to upload files directly in Colab
# Skip this cell if you're using Google Drive or local files
# ============================================================================

if IN_COLAB:
    try:
        import ipywidgets as widgets
        from IPython.display import display
        
        print("📤 File Upload Widget")
        print("=" * 60)
        print("Click 'Upload' to select files from your computer.")
        print("Uploaded files will be saved to /content/data/")
        print("=" * 60)
        
        # Create upload button
        upload_button = widgets.FileUpload(
            accept='',  # Accept all file types
            multiple=True,
            description='Upload Files'
        )
        
        def handle_upload(change):
            """Handle file uploads"""
            for filename, file_info in upload_button.value.items():
                # Save uploaded file
                file_path = os.path.join(WORK_DIR, filename)
                with open(file_path, 'wb') as f:
                    f.write(file_info['content'])
                print(f"✓ Uploaded: {filename} -> {file_path}")
        
        upload_button.observe(handle_upload, names='value')
        display(upload_button)
        
        print("\n💡 Tip: After uploading, check /content/data/ for your files")
        print("   Then update file paths in Cell 3 (Configuration)")
    except ImportError:
        print("📤 Alternative: Use files.upload() from google.colab")
        print("   Run: from google.colab import files; files.upload()")
        print("   This will prompt you to select files to upload")
else:
    print("⏭ Skipped (not running in Colab)")

In [ ]:
# Cell 1: Install Dependencies
# ============================================================================
# Install vdapseisutils and required packages
# ============================================================================

# Upgrade pip first (helps avoid installation issues in Colab)
!pip install --upgrade pip -q

# Option 1: Try installing from requirements.txt if it exists
# (Uncomment and modify the URL if vdapseisutils has a requirements.txt)
# !wget -q https://raw.githubusercontent.com/jwellik/vdapseisutils/main/requirements.txt
# !pip install -r requirements.txt -q

# Option 2: Install vdapseisutils from GitHub
# This will install the package and its dependencies (if defined in setup.py/pyproject.toml)
!pip install git+https://github.com/jwellik/vdapseisutils.git -q

# Install core dependencies explicitly (in case they're not auto-installed)
# These are the main packages used by vdapseisutils and this notebook
!pip install numpy scipy matplotlib obspy pandas ipywidgets -q

print("✓ Dependencies installed successfully!")
print("\n💡 Note: If you encounter import errors, restart the runtime:")
print("   Runtime → Restart runtime (or use Ctrl+M .)")

In [ ]:
# Cell 2: Import Libraries
# ============================================================================
import numpy as np
import matplotlib.pyplot as plt
from obspy import read, UTCDateTime, read_events
from obspy.core.event import Event
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import vdapseisutils components
from vdapseisutils.dev.buurman_2010 import Buurman_2010
from vdapseisutils import VCatalog
from vdapseisutils.core.datasource import DataSource

print("✓ Libraries imported successfully!")

In [ ]:
# Cell 3: Configuration - Input Files and Parameters
# ============================================================================
# Define all input file paths and parameters here
# ============================================================================

# ========== INPUT FILES ==========
# For OPTION A (Compute FI from catalog + datasource):
catalog_file = None  # Path to catalog file (QuakeML, CSV, etc.)
# Examples:
#   Colab: catalog_file = "/content/data/catalog.xml"
#   Drive: catalog_file = "/content/drive/MyDrive/catalog.xml"
#   Local: catalog_file = "/path/to/catalog.xml"
#   URL:   catalog_file = "https://example.com/catalog.xml"  # (download first)

# For OPTION B (Load pre-computed data):
precomputed_data_file = None  # Path to CSV with times, FI, magnitudes
# Examples:
#   Colab: precomputed_data_file = "/content/data/data.csv"
#   Drive: precomputed_data_file = "/content/drive/MyDrive/data.csv"
#   Local: precomputed_data_file = "/path/to/data.csv"

# For waveform files (if loading from files instead of datasource):
waveform_directory = None  # Directory containing waveform files
# Examples:
#   Colab: waveform_directory = "/content/data/waveforms/"
#   Drive: waveform_directory = "/content/drive/MyDrive/waveforms/"
#   Local: waveform_directory = "/path/to/waveforms/"

# For example waveform (used in panels A-C of the figure):
example_waveform_file = None  # Path to example waveform file
# Examples:
#   Colab: example_waveform_file = "/content/data/example_waveform.mseed"
#   Drive: example_waveform_file = "/content/drive/MyDrive/example_waveform.mseed"
#   Local: example_waveform_file = "/path/to/example_waveform.mseed"

# ========== ANALYSIS PARAMETERS ==========
# Frequency bands for Frequency Index calculation
Alower = [1, 2.5]      # Lower frequency band [Hz]
Aupper = [5, 10]        # Upper frequency band [Hz]

# Analysis window (time window in seconds [start, end] for spectrum/FI calculation)
# Set to None to use entire waveform
analysis_window = [1, 5]  # or None

# FI classification thresholds (from Buurman and West, 2010)
# Events are classified as:
#   - Low frequency: FI < lowest threshold
#   - Hybrid: FI between thresholds  
#   - High frequency: FI > highest threshold
# Set to None to disable automatic color coding
thresholds = [-1.3, -0.4]  # or None

# Colors for classification regions (one more than thresholds)
threshold_colors = ["black", "red", "blue"]  # [low, hybrid, high] or None

# Highlight color for frequency bands and analysis window
highlight_color = "lightgreen"
highlight_alpha = 0.3

# ========== DATASOURCE CONFIGURATION ==========
# For OPTION A: Configure waveform datasource
# Option 1: FDSN client (e.g., IRIS)
datasource_string = "IRIS"  # or None to skip datasource setup

# Option 2: Earthworm/Winston waveserver
# datasource_string = "127.0.0.1:16022"

# Option 3: SDS filesystem
# datasource_string = "sds:///path/to/sds/archive"

# ========== WORKFLOW SELECTION ==========
# Set which workflow to use:
COMPUTE_FI = True   # True: Compute FI from catalog + datasource (OPTION A)
LOAD_OWN_DATA = False  # True: Load pre-computed data (OPTION B)
# Note: Only one should be True at a time

# ========== WAVEFORM DOWNLOAD PARAMETERS ==========
# (Only used if COMPUTE_FI = True)
pre_t = 2      # Seconds before origin time
post_t = 18    # Seconds after origin time
components = ["Z"]  # Components to download

print("✓ Configuration loaded!")
print(f"  Workflow: {'Compute FI' if COMPUTE_FI else 'Load pre-computed data'}")
print(f"  Frequency bands: {Alower} - {Aupper} Hz")
print(f"  Analysis window: {analysis_window}")

# Helper function to download files from URL (useful in Colab)
def download_file_from_url(url, save_path=None):
    """Download a file from URL (useful in Colab)"""
    import urllib.request
    if save_path is None:
        save_path = os.path.join(WORK_DIR, os.path.basename(url))
    urllib.request.urlretrieve(url, save_path)
    print(f"✓ Downloaded: {url} -> {save_path}")
    return save_path

# Example: Download a file from URL (uncomment and modify as needed)
# if catalog_file and catalog_file.startswith('http'):
#     catalog_file = download_file_from_url(catalog_file)

## Option A: Compute FI from Catalog and Waveform DataSource

In [ ]:
# Cell 4: OPTION A - Compute FI from Catalog and Waveform DataSource
# ============================================================================
# Skip this cell if LOAD_OWN_DATA = True (go to Cell 5)
# ============================================================================

if COMPUTE_FI:
    # ========== Load Catalog ==========
    if catalog_file is None:
        raise ValueError("Please set 'catalog_file' in Cell 3")
    
    # Check if file exists
    if not os.path.exists(catalog_file):
        raise FileNotFoundError(
            f"Catalog file not found: {catalog_file}\n"
            f"Please upload the file or check the path.\n"
            f"In Colab, use the file upload widget or mount Google Drive."
        )
    
    # Load catalog (supports QuakeML format)
    print(f"Loading catalog from: {catalog_file}")
    catalog = read_events(catalog_file)
    catalog = VCatalog(catalog)
    
    print(f"✓ Loaded catalog with {len(catalog)} events")
    
    # ========== Set up Waveform DataSource ==========
    if datasource_string is None:
        raise ValueError("Please set 'datasource_string' in Cell 3")
    
    datasource = DataSource(datasource_string)
    print(f"✓ DataSource configured: {datasource.name}")
    
    # ========== Download Waveforms ==========
    print("Downloading waveforms...")
    event_streams = catalog.get_waveforms(
        client=datasource.client,
        pre_t=pre_t,
        post_t=post_t,
        components=components,
        verbose=True
    )
    
    # event_streams is a dict: {event_id: Stream}
    print(f"✓ Downloaded waveforms for {len(event_streams)} events")
    
    # ========== Extract Traces for FI Calculation ==========
    # For each event, extract the trace you want to use for FI calculation
    # This example uses the first Z-component trace from each event
    event_traces = []
    for event in catalog:
        event_id = event.resource_id.id if hasattr(event.resource_id, 'id') else str(event.resource_id)
        if event_id in event_streams:
            stream = event_streams[event_id]
            # Get Z component (or modify to select specific station/channel)
            z_traces = [tr for tr in stream if tr.stats.channel[-1] == 'Z']
            if z_traces:
                event_traces.append(z_traces[0])  # Use first Z trace
            else:
                event_traces.append(None)
        else:
            event_traces.append(None)
    
    # Filter out events without traces
    valid_indices = [i for i, tr in enumerate(event_traces) if tr is not None]
    catalog_filtered = VCatalog([catalog[i] for i in valid_indices])
    event_traces_filtered = [event_traces[i] for i in valid_indices]
    
    print(f"✓ Extracted traces for {len(event_traces_filtered)} events")
    
    # ========== Calculate FI Values ==========
    # Use Buurman_2010.extract_catalog_data() to compute FI for all events
    times, fi_values, magnitudes = Buurman_2010.extract_catalog_data(
        catalog=catalog_filtered,
        event_traces=event_traces_filtered,
        Alower=Alower,
        Aupper=Aupper,
        analysis_window=analysis_window
    )
    
    # Store waveforms for later use (optional - for plotting example waveform)
    waveforms = event_traces_filtered
    
    # Store catalog for potential map plotting
    catalog_for_plotting = catalog_filtered
    
    print(f"✓ Computed FI values for {len(fi_values)} events")
    print(f"  FI range: {min(fi_values):.2f} to {max(fi_values):.2f}")
    if magnitudes and any(m is not None for m in magnitudes):
        valid_mags = [m for m in magnitudes if m is not None]
        print(f"  Magnitude range: {min(valid_mags):.2f} to {max(valid_mags):.2f}")
    
    print("\n✓ FI computation complete! Data ready for plotting.")
else:
    print("⏭ Skipped FI computation. Proceed to Cell 5 to load your own data.")

## Option B: Load Pre-computed Data

In [ ]:
# Cell 5: OPTION B - Load Pre-computed Data
# ============================================================================
# Use this cell if you already have computed FI values, magnitudes, and waveforms
# Skip this cell if COMPUTE_FI = True
# ============================================================================

if LOAD_OWN_DATA:
    # ========== Option 1: Load from CSV file ==========
    if precomputed_data_file is None:
        raise ValueError("Please set 'precomputed_data_file' in Cell 3")
    
    # Check if file exists
    if not os.path.exists(precomputed_data_file):
        raise FileNotFoundError(
            f"Data file not found: {precomputed_data_file}\n"
            f"Please upload the file or check the path.\n"
            f"In Colab, use the file upload widget or mount Google Drive."
        )
    
    print(f"Loading data from: {precomputed_data_file}")
    df = pd.read_csv(precomputed_data_file)
    
    # Expected columns: 'time', 'FI' (or 'fi', 'frequency_index'), 'magnitude' (optional)
    # Convert time column to UTCDateTime
    times = [UTCDateTime(t) for t in df['time']]
    
    # Try different possible column names for FI
    if 'FI' in df.columns:
        fi_values = df['FI'].values
    elif 'fi' in df.columns:
        fi_values = df['fi'].values
    elif 'frequency_index' in df.columns:
        fi_values = df['frequency_index'].values
    else:
        raise ValueError("CSV must contain 'FI', 'fi', or 'frequency_index' column")
    
    magnitudes = df['magnitude'].values if 'magnitude' in df.columns else None
    
    # Load waveforms (if available)
    waveforms = None
    if 'waveform_file' in df.columns:
        waveforms = [read(f)[0] for f in df['waveform_file']]
    elif waveform_directory is not None:
        # Load from directory based on times
        waveforms = []
        for t in times:
            # Adjust pattern based on your file naming convention
            filename = f"{waveform_directory}/{t.strftime('%Y%m%d_%H%M%S')}.mseed"
            try:
                waveforms.append(read(filename)[0])
            except:
                waveforms.append(None)
        waveforms = [w for w in waveforms if w is not None] if waveforms else None
    
    # Load catalog if available (for potential map plotting)
    catalog_for_plotting = None
    if catalog_file is not None:
        try:
            if os.path.exists(catalog_file):
                catalog_for_plotting = VCatalog(read_events(catalog_file))
            else:
                print(f"Warning: Catalog file not found: {catalog_file}")
        except Exception as e:
            print(f"Warning: Could not load catalog file: {e}")
    
    print(f"✓ Loaded {len(times)} events from CSV")
    print(f"  FI range: {min(fi_values):.2f} to {max(fi_values):.2f}")
    if magnitudes is not None:
        valid_mags = [m for m in magnitudes if m is not None]
        if valid_mags:
            print(f"  Magnitude range: {min(valid_mags):.2f} to {max(valid_mags):.2f}")
    
    print("\n✓ Data loaded! Ready for plotting.")
else:
    print("⏭ Skipped loading own data. Using data from Cell 4.")

## Create Figures

In [ ]:
# Cell 6: Create Buurman_2010 Instance and Add Data
# ============================================================================

# Create Buurman_2010 instance with configuration
buurman = Buurman_2010(
    Alower=Alower,
    Aupper=Aupper,
    thresholds=thresholds,
    threshold_colors=threshold_colors,
    analysis_window=analysis_window,
    highlight_color=highlight_color,
    highlight_alpha=highlight_alpha
)

# Add catalog data (times, FI values, magnitudes)
buurman.add_catalog(
    times=times,
    fi_values=fi_values,
    magnitudes=magnitudes,
    marker='o',
    facecolor='lightgrey',
    alpha=0.7,
    edgecolor='k',
    s=50  # marker size
)

print("✓ Buurman_2010 instance created and data added")

In [ ]:
# Cell 7: Create Comprehensive Figure
# ============================================================================
# This creates a figure with:
# - Upper left: Waveform, spectrum, and frequency index (panels A-C)
# - Upper right: FI vs Magnitude scatter plot (panel D)
# - Bottom: Timeseries of FI over time (panel E)

# Select example waveform for panels A-C
# Option 1: Use first waveform from computed/loaded data
if 'waveforms' in locals() and waveforms and len(waveforms) > 0:
    example_waveform = waveforms[0]
# Option 2: Load from configured file path
elif example_waveform_file is not None:
    example_waveform = read(example_waveform_file)[0]
# Option 3: Use None to skip waveform panels
else:
    example_waveform = None

# Create the comprehensive figure
fig = buurman.plot_all(
    figsize=(14, 10),
    example_waveform=example_waveform,
    title_waveform="EXAMPLE WAVEFORM—SPECTRUM—FI",
    title_timeseries="TIME SERIES OF FREQUENCY INDEX"
)

# Optionally set custom axis limits
# buurman.set_spectra_xlim([0.5, 25])  # Frequency range for spectrum plot
# buurman.set_fi_ylim([-2.5, 1.0])     # FI range for timeseries plot

plt.tight_layout()
plt.show()

print("✓ Figure created and displayed")

In [ ]:
# Cell 8: Save Figure (Optional)
# ============================================================================

# Save as PNG
# buurman.save("buurman_figure.png", dpi=300)

# Save as SVG (vector format)
# buurman.save("buurman_figure.svg", dpi=300)

print("✓ Save figure (commented out - uncomment to run)")

## Alternative: Individual Figure Types

In [ ]:
# Cell 8b: Download Figure in Colab (Optional)
# ============================================================================
# Use this cell to download figures directly in Google Colab
# ============================================================================

if 'IN_COLAB' in globals() and IN_COLAB:
    # Uncomment the lines below to save and download the figure:
    # from google.colab import files
    # fig.savefig("/content/buurman_figure.png", dpi=300, bbox_inches='tight')
    # files.download("/content/buurman_figure.png")
    print("💡 To download figures in Colab, uncomment the code above")
else:
    print("⏭ Skipped (not running in Colab)")

In [ ]:
# Cell 9: Alternative - Individual Figure Types
# ============================================================================
# Create individual figure types instead of the comprehensive plot

# Figure 2: Waveform only
# fig2 = buurman.plot_fig2(figsize=(8, 4))

# Figure 3: Spectra and frequency index
# fig3 = buurman.plot_fig3(figsize=(8, 6))

# Figure 23: Combined waveform, spectra, and frequency index
# fig23 = buurman.plot_fig23(figsize=(8, 8))

# Figure 4: Timeseries of frequency index
# fig4 = buurman.plot_fig4(figsize=(8, 6))

# FI vs Magnitude scatter plot
# fig_mag = buurman.plot_fi_magnitude(figsize=(6, 6))

# plt.show()